In [ ]:
"""
import os
import numpy as np
import librosa
from joblib import load
from tensorflow.keras.models import load_model
"""

In [ ]:
"""
# Adjust relative paths based on notebook location
SCALER_PATH = "D:/University/AI/Project/Alzheimer Speech Detection/models/scalar.joblib"
RF_MODEL_PATH = "D:/University/AI/Project/Alzheimer Speech Detection/models/rf_alz_model.joblib"
# NN_MODEL_PATH = "D:/University/AI/Project/Alzheimer Speech Detection/models/nn_alz_model.h5"

# Load the models
scaler = load(SCALER_PATH)
rf_model = load(RF_MODEL_PATH)
# nn_model = load_model(NN_MODEL_PATH)

print("✅ Models and scaler loaded successfully!")
"""

✅ Models and scaler loaded successfully!


In [ ]:
"""
def extract_features(y, sr):
    # Extracts a variety of speech features from an audio signal.
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    spec_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr), axis=1)

    features = np.hstack([mfccs, chroma, [zcr], spec_contrast])
    return features
    """


In [ ]:
"""# Provide the path to your test audio file (WAV format)
AUDIO_FILE = "D:/University/AI/Project/Alzheimer Speech Detection/data/audio_files/19-198-0007_impaired.wav"

y, sr = librosa.load(AUDIO_FILE, sr=22050)
print(f"✅ Loaded audio file with {len(y)} samples at {sr} Hz")
"""


✅ Loaded audio file with 374889 samples at 22050 Hz


In [ ]:
"""
# Step 5: Extract features
features = extract_features(y, sr)

# The model was trained on 65 features (13 MFCC + 52 linguistic)
expected_features = 65

if len(features) < expected_features:
    features = np.pad(features, (0, expected_features - len(features)), 'constant')
elif len(features) > expected_features:
    features = features[:expected_features]

# Scale the features
features_scaled = scaler.transform([features])

print(f"✅ Feature vector shape after padding/scaling: {features_scaled.shape}")
"""


✅ Feature vector shape after padding/scaling: (1, 65)


c:\Users\m\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


It checks how many features your extracted vector has.

If it has fewer (33), it pads with zeros → now you have exactly 65.

The model + scaler can now handle the input perfectly.

No retraining needed.

In [95]:
"""
# =============================
# Step 6 — Make Predictions
# =============================
import json

# --- Load model accuracies ---
with open("D:/University/AI/Project/Alzheimer Speech Detection/models/accuracies.json", "r") as f:
    accuracies = json.load(f)

rf_accuracy = accuracies["random_forest"]
nn_accuracy = accuracies["neural_network"]

# --- Random Forest prediction ---
rf_pred = rf_model.predict(features_scaled)[0]
rf_confidence = rf_model.predict_proba(features_scaled)[0][1]  # Probability of Alzheimer

# --- Neural Network prediction ---
nn_prob = nn_model.predict(features_scaled)[0][0]
nn_pred = int(nn_prob > 0.5)

label_map = {0: "Healthy", 1: "Alzheimer"}

print("🧩 Random Forest Prediction:", label_map[rf_pred], f"(Confidence: {rf_confidence:.2f})")
print("🧠 Neural Network Prediction:", label_map[nn_pred], f"(Confidence: {nn_prob:.2f})")

# --- Automatically choose the more accurate model ---
if rf_accuracy >= nn_accuracy:
    final_model = "Random Forest"
    final_pred = rf_pred
else:
    final_model = "Neural Network"
    final_pred = nn_pred

print(f"\n🏆 FINAL DECISION (Based on {final_model} - higher accuracy): {label_map[final_pred]}")

"""

'\n# =============================\n# Step 6 — Make Predictions\n# =============================\nimport json\n\n# --- Load model accuracies ---\nwith open("D:/University/AI/Project/Alzheimer Speech Detection/models/accuracies.json", "r") as f:\n    accuracies = json.load(f)\n\nrf_accuracy = accuracies["random_forest"]\nnn_accuracy = accuracies["neural_network"]\n\n# --- Random Forest prediction ---\nrf_pred = rf_model.predict(features_scaled)[0]\nrf_confidence = rf_model.predict_proba(features_scaled)[0][1]  # Probability of Alzheimer\n\n# --- Neural Network prediction ---\nnn_prob = nn_model.predict(features_scaled)[0][0]\nnn_pred = int(nn_prob > 0.5)\n\nlabel_map = {0: "Healthy", 1: "Alzheimer"}\n\nprint("🧩 Random Forest Prediction:", label_map[rf_pred], f"(Confidence: {rf_confidence:.2f})")\nprint("🧠 Neural Network Prediction:", label_map[nn_pred], f"(Confidence: {nn_prob:.2f})")\n\n# --- Automatically choose the more accurate model ---\nif rf_accuracy >= nn_accuracy:\n    final_mo

In [ ]:
'''
# =============================
# Step 6 — Make Predictions
# =============================

# --- Random Forest prediction ---
rf_prob = rf_model.predict_proba(features_scaled)[0][1]  # probability of Alzheimer
rf_threshold = 0.6  # adjust based on validation
rf_pred = int(rf_prob > rf_threshold)

""" ---
# --- Neural Network prediction ---
nn_prob = nn_model.predict(features_scaled)[0][0]  # probability of Alzheimer
nn_threshold = 0.6
nn_pred = int(nn_prob > nn_threshold)
--- """

# --- Label mapping ---
label_map = {0: "Healthy", 1: "Alzheimer"}

print(f"🧩 Random Forest Prediction: {label_map[rf_pred]} (Confidence: {rf_prob:.2f})")
# print(f"🧠 Neural Network Prediction: {label_map[nn_pred]} (Confidence: {nn_prob:.2f})")

""" ---
# --- Final decision ---
if rf_pred != nn_pred:
    final_pred = rf_pred if rf_prob > nn_prob else nn_pred
else:
    final_pred = rf_pred
---"""


'''



🧩 Random Forest Prediction: Healthy (Confidence: 0.49)


'\n# --- Final decision ---\nif rf_pred != nn_pred:\n    final_pred = rf_pred if rf_prob > nn_prob else nn_pred\nelse:\n    final_pred = rf_pred\n\n'

In [108]:
# =============================
# Step 1 — Imports
# =============================
import os
import numpy as np
import librosa
from joblib import load
from tensorflow.keras.models import load_model

In [109]:
# =============================
# Step 2 — Load Models and Scaler
# =============================
MODEL_DIR = "D:/University/AI/Project/Alzheimer Speech Detection/models"
scaler = load(os.path.join(MODEL_DIR, "scaler.joblib"))
rf_model = load(os.path.join(MODEL_DIR, "rf_alz_model.joblib"))
nn_model = load_model(os.path.join(MODEL_DIR, "nn_alz_model.h5"))

In [110]:
# =============================
# Step 3 — Feature Extraction
# =============================
def extract_features(y, sr):
    """
    Extract 65 features from audio: 13 MFCC + 52 linguistic
    Replace the linguistic features part with your implementation
    """
    # --- MFCCs ---
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)

    # --- Placeholder for 52 linguistic features ---
    linguistic_features = np.zeros(52)  # Replace with real extraction if available

    features = np.concatenate([mfccs_mean, linguistic_features])
    
    # Pad or truncate to ensure length = 65
    expected_len = 65
    if len(features) < expected_len:
        features = np.pad(features, (0, expected_len - len(features)), 'constant')
    elif len(features) > expected_len:
        features = features[:expected_len]
    
    return features

In [111]:

# =============================
# Step 4 — Ensemble Prediction
# =============================
def predict_alzheimer_from_audio(audio_path, scaler, rf_model, nn_model, rf_threshold=0.6, nn_threshold=0.6):
    # Load audio
    y, sr = librosa.load(audio_path, sr=22050)
    
    # Extract features
    features = extract_features(y, sr)
    
    # Scale features
    features_scaled = scaler.transform([features])
    
    # Random Forest
    rf_prob = rf_model.predict_proba(features_scaled)[0][1]
    rf_pred = int(rf_prob > rf_threshold)
    
    # Neural Network
    nn_prob = nn_model.predict(features_scaled)[0][0]
    nn_pred = int(nn_prob > nn_threshold)
    
    # Ensemble: majority vote with confidence tie-break
    if rf_pred != nn_pred:
        final_pred = rf_pred if rf_prob > nn_prob else nn_pred
    else:
        final_pred = rf_pred
    
    label_map = {0: "Healthy", 1: "Alzheimer"}
    return label_map[final_pred], rf_prob, nn_prob

In [125]:
# =============================
# Step 5 — Example Usage
# =============================

audio_file = "D:/University/AI/Project/Alzheimer Speech Detection/data/audio_files/19-198-0000.wav"  # replace with your audio path
final_pred, rf_prob, nn_prob = predict_alzheimer_from_audio(audio_file, scaler, rf_model, nn_model)

# Individual model predictions
rf_pred = "Alzheimer" if rf_prob > 0.6 else "Healthy"
nn_pred = "Alzheimer" if nn_prob > 0.6 else "Healthy"

print()

print(f"Random Forest Prediction: {rf_pred} (Confidence: {rf_prob:.2f})")
print(f"Neural Network Prediction: {nn_pred} (Confidence: {nn_prob:.2f})")
print()
print(f"Ensemble Final Prediction: {final_pred}")

c:\Users\m\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step

Random Forest Prediction: Healthy (Confidence: 0.57)
Neural Network Prediction: Healthy (Confidence: 0.33)

Ensemble Final Prediction: Healthy


# 🔍 Explanation

1. **Feature Extraction** — We compute MFCC, chroma, spectral contrast, and zero-crossing features from the input `.wav` file.  
2. **Feature Padding** — Since the original model was trained on 65 features (MFCC + linguistic), we pad or trim the feature vector to match that length.  
3. **Scaling** — The features are normalized using the same `StandardScaler` used during training.  
4. **Prediction** — Both Random Forest and Neural Network models predict whether the speech indicates Alzheimer’s or not.  
5. **Output** — The notebook prints the classification (“Healthy” or “Alzheimer”).  
